# Résolution du problème de tournées de véhicules (VRP) avec OR-Tools et OSRM

## Étape 1 : Chargement des données des clients

In [23]:
import pandas as pd

# Charger le fichier xlsx
clients_df = pd.read_excel("clients/casa-nord.xlsx")

# Renommer les colonnes pour correspondre au format attendu
clients_df = clients_df.rename(columns={
    'PARTNER_CODE': 'id',
    'LATITUDE': 'lat', 
    'LONGITUDE': 'lng',
    'WEIGHT': 'weight'
})

# Ajouter le dépôt (à Casablanca) au début
depot = pd.DataFrame([{
    'id': 'DEPOT_CASA', 
    'lat': 33.604427, 
    'lng': -7.558631, 
    'weight': 0
}])

clients_df = pd.concat([depot, clients_df], ignore_index=True)

# Supprimer les points avec des coordonnées == 0
clients_df = clients_df[~((clients_df['lat'] == 0) & (clients_df['lng'] == 0))]
# Réinitialiser les index
clients_df = clients_df.reset_index(drop=True)

# Afficher les premières lignes
print(clients_df.head())

print(f"\nNombre total de points (dépôt + clients): {len(clients_df)}")
print(f"Poids total à livrer: {clients_df['weight'][1:].sum():.2f} kg -- {clients_df['weight'][1:].sum() / 1000:.2f} tonnes")


           id        lat       lng  weight
0  DEPOT_CASA  33.604427 -7.558631    0.00
1      C36713  33.586656 -7.581679   18.82
2      C33753  33.591718 -7.556250    3.16
3     CM00493  33.600761 -7.629347   25.48
4      C26049  33.685130 -7.408309   37.72

Nombre total de points (dépôt + clients): 2769
Poids total à livrer: 62888.61 kg -- 62.89 tonnes


## Étape 2 : Calcul de la matrice de distances et de temps avec OSRM

In [1]:
import requests
from itertools import combinations

# Nombre total de points (dépôt + clients)
N = len(clients_df)

# Initialiser la matrice de distances avec des zéros (en km)
distance_matrix = [[0 for _ in range(N)] for _ in range(N)]

# Initialiser la matrice de temps (en minutes)
time_matrix = [[0 for _ in range(N)] for _ in range(N)]

# Définir la taille des lots (réduite à 50 pour éviter les limites d’OSRM)
batch_size = 50

# Créer des lots d’indices de clients (1 à N-1, car 0 est le dépôt)
client_indices = list(range(1, N))
batches = [client_indices[i:i + batch_size] for i in range(0, len(client_indices), batch_size)]

# Fonction pour valider les coordonnées
def validate_coords(lng, lat):
    # Validation de base : vérifier si les coordonnées sont dans des limites raisonnables
    return -180 <= lng <= 180 and -90 <= lat <= 90

# Fonction pour faire la requête OSRM et retourner les sous-matrices distance (km) et temps (min)
def get_sub_matrices(points):
    # Extraire et valider les coordonnées
    coords_list = []
    for p in points:
        lng, lat = clients_df['lng'].iloc[p], clients_df['lat'].iloc[p]
        if not validate_coords(lng, lat):
            print(f"Coordonnées invalides pour le point {p} : ({lat}, {lng})")
            return None, None
        coords_list.append(f"{lng},{lat}")

    coords = ";".join(coords_list)
    url = f"http://localhost:5001/table/v1/driving/{coords}?annotations=distance,duration"
    
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            sub_distances = [[round(d / 1000, 2) if d is not None else 0 for d in row] for row in data['distances']]  # km
            sub_durations = [[round(t / 60, 2) if t is not None else 0 for t in row] for row in data['durations']]  # min
            return sub_distances, sub_durations
        else:
            print(f"Erreur OSRM : {response.status_code} pour les points {points}")
            print(f"Réponse : {response.text}")
            return None, None
    except requests.exceptions.RequestException as e:
        print(f"Échec de la requête pour les points {points} : {e}")
        return None, None

# Étape 1 : Requêtes pour le dépôt + chaque lot
for batch_idx, batch in enumerate(batches):
    print(f"Traitement du lot {batch_idx + 1}/{len(batches)}")
    points = [0] + batch  # Inclure le dépôt (index 0) + le lot
    sub_distances, sub_durations = get_sub_matrices(points)
    if sub_distances is None:
        print(f"Lot {batch_idx + 1} ignoré à cause d’une erreur")
        continue
    
    # r est l’index d’une ligne (le point de départ)
    # c est l’index d’une colonne (le point d’arrivée)
    for r in range(len(points)):
        for c in range(len(points)):
            if r == 0 and c == 0:
                distance_matrix[0][0] = sub_distances[r][c]  # Dépôt vers dépôt
                time_matrix[0][0] = sub_durations[r][c]  # Temps dépôt vers dépôt
            elif r == 0:
                distance_matrix[0][points[c]] = sub_distances[r][c]  # Dépôt vers client
                time_matrix[0][points[c]] = sub_durations[r][c]  # Temps dépôt vers client
            elif c == 0:
                distance_matrix[points[r]][0] = sub_distances[r][c]  # Client vers dépôt
                time_matrix[points[r]][0] = sub_durations[r][c]  # Temps client vers dépôt
            else:
                distance_matrix[points[r]][points[c]] = sub_distances[r][c]  # Entre clients du même lot
                time_matrix[points[r]][points[c]] = sub_durations[r][c]  # Entre clients du même lot

# Étape 2 : Requêtes pour chaque paire de lots
# Utiliser des combinaisons pour éviter les doublons 
# (On utilise combinations pour ne pas répéter (1,2) et (2,1), car la distance est la même)
for b1_idx, b2_idx in combinations(range(len(batches)), 2):
    b1, b2 = batches[b1_idx], batches[b2_idx]
    print(f"Traitement de la paire de lots ({b1_idx + 1}, {b2_idx + 1})")
    points = b1 + b2
    sub_distances, sub_durations = get_sub_matrices(points)
    if sub_distances is None:
        print(f"Paire de lots ({b1_idx + 1}, {b2_idx + 1}) ignorée à cause d’une erreur")
        continue
    
    len_b1 = len(b1)
    for i in range(len_b1):
        for j in range(len(b2)):
            distance_matrix[b1[i]][b2[j]] = sub_distances[i][len_b1 + j]  # b1 vers b2
            distance_matrix[b2[j]][b1[i]] = sub_distances[len_b1 + j][i]  # b2 vers b1 (supposé symétrique)

            time_matrix[b1[i]][b2[j]] = sub_durations[i][len_b1 + j]  # Temps b1 vers b2
            time_matrix[b2[j]][b1[i]] = sub_durations[len_b1 + j][i]  # Temps b2 vers b1 (supposé symétrique)

# Vérifier si la matrice de distances est complètement remplie
non_zero_count = sum(1 for row in distance_matrix for d in row if d != 0)
# Verifier si la matrice de temps est complètement remplie
non_zero_time_count = sum(1 for row in time_matrix for t in row if t != 0)
print(f"Matrice de distances remplie avec {non_zero_count} entrées non nulles")
print(f"Matrice de temps remplie avec {non_zero_time_count} entrées non nulles")
print(f"Exemple (dépôt -> premier client) : {distance_matrix[0][1]} km, {time_matrix[0][1]} min")


NameError: name 'clients_df' is not defined

In [ ]:
# Afficher la matrice de distances
distance_df = pd.DataFrame(distance_matrix, index=clients_df.index, columns=clients_df.index)
print(distance_df)

# Notez que la matrice n'est pas parfaitement symétrique (par exemple, distance_df.iloc[0, 1] ≠ distance_df.iloc[1, 0]), car OSRM calcule les distances en fonction des routes réelles, qui peuvent différer légèrement

       0      1      2      3      4      5      6      7      8      9     \
0      0.00   6.27   3.61   8.20  17.41   6.57   4.07   7.05   6.85  22.89   
1      5.20   0.00   2.99   5.40  23.48  11.52   3.05   3.10   6.36  26.12   
2      3.94   3.23   0.00   8.36  21.67   7.26   0.87   3.83   5.59  24.31   
3      8.68   5.97   8.89   0.00  25.03  15.63   8.71   7.84  12.03  28.09   
4     19.93  22.90  20.59  24.48   0.00  14.35  20.58  22.75  21.53   3.43   
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
2764   7.65  10.21   8.31  15.35  15.54   3.55   9.05  10.07   5.29  18.18   
2765   7.28   5.28   5.86  10.42  23.82  10.30   4.87   4.79   2.29  26.46   
2766  27.49  30.46  28.15  35.59  13.62  21.91  28.14  30.31  29.09   4.57   
2767   4.78   4.19   3.36   9.33  19.77   7.80   2.37   4.05   2.58  22.40   
2768  14.96   9.97  12.89   5.91  31.31  21.91  12.72  11.76  16.03  34.37   

      ...   2759   2760   2761   2762   2763   2764   2765   27

In [ ]:
# Afficher la matrice de temps
time_df = pd.DataFrame(time_matrix, index=clients_df.index, columns=clients_df.index)
print(time_df)

# Notez que la matrice de temps n'est pas parfaitement symétrique non plus (par exemple, time_df.iloc[0, 1] ≠ time_df.iloc[1, 0]), car les temps de trajet peuvent varier en fonction des conditions de circulation et des routes empruntées

       0      1      2      3      4      5      6      7      8      9     \
0      0.00   8.31   5.47  10.37  19.58   9.16   6.06   8.43   8.62  24.14   
1      8.35   0.00   5.17   6.76  21.81  12.71   4.31   3.73   7.82  26.02   
2      5.85   5.07   0.00  11.03  20.65   9.94   2.48   5.91   7.40  24.86   
3     10.44   7.40  11.68   0.00  25.68  17.11  10.74   9.15  14.25  30.30   
4     18.92  20.40  19.73  25.62   0.00  12.63  18.70  20.48  18.28   5.32   
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
2764  10.50  13.12  11.31  19.08  14.32   5.50  11.51  13.20   7.33  18.53   
2765   9.16   7.28   7.91  13.24  19.76  12.01   6.78   6.75   3.03  23.97   
2766  23.40  24.88  24.21  30.84  10.19  17.11  23.18  24.95  22.76   6.46   
2767   6.33   5.56   5.08  11.52  18.29   9.18   3.95   5.63   3.57  22.50   
2768  16.75  12.84  17.12   7.99  32.00  23.43  16.18  14.62  19.68  36.62   

      ...   2759   2760   2761   2762   2763   2764   2765   27

## Étape 3 : Résolution du VRP avec OR-Tools

In [ ]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
# time pour mesurer le temps d'exécution
import time
# weakref pour garder une référence faible à l'objet manager et routing_model pour éviter les fuites de mémoire
import weakref

# Extraire les demandes en kg -> grammes (int)
# Si une ligne est vide, on met 0
demands = (clients_df['weight'].fillna(0) * 1000).astype(int).tolist()

# Définir les paramètres du VRP
num_vehicles = 36
depot_index = 0
vehicle_capacities = [4_000_000] * num_vehicles  # 4 tonnes en grammes

# Dictionnaire de données
data = {
    'distance_matrix': distance_matrix,   # km
    'time_matrix': time_matrix,           # minutes
    'demands': demands,                   # grammes
    'vehicle_capacities': vehicle_capacities,
    'num_vehicles': num_vehicles,
    'depot': depot_index,
}

# Créer l'index manager
manager = pywrapcp.RoutingIndexManager(
    len(data['distance_matrix']),
    data['num_vehicles'],
    data['depot']
)

# Créer le modèle de routage
routing = pywrapcp.RoutingModel(manager)

# Callback interactif à chaque solution
class InteractiveSolutionCallback:
    """
    Appelé par OR-Tools à chaque solution; affiche distance totale et nb de véhicules,
    puis propose d'arrêter ou continuer. Filtré toutes les 'interval_seconds'.
    """
    def __init__(self, manager, routing_model, interval_seconds=60):
        self._mgr_ref = weakref.ref(manager)
        self._routing_ref = weakref.ref(routing_model)
        self._interval = interval_seconds
        self._last = 0.0

    def __call__(self):
        now = time.time()
        if now - self._last < self._interval:
            return
        self._last = now

        routing_model = self._routing_ref()
        manager = self._mgr_ref()

        # Objectif courant (mètres, car on convertit km -> m dans le coût d’arc)
        try:
            total_cost_m = int(routing_model.CostVar().Value())
            # Soustraire les coûts fixes des véhicules pour obtenir seulement la distance
            vehicles_used = 0
            for v in range(manager.GetNumberOfVehicles()):
                start = routing_model.Start(v)
                if not routing_model.IsEnd(routing_model.NextVar(start).Value()):
                    vehicles_used += 1
            total_distance_only = total_cost_m - (vehicles_used * fixed_cost)
        except Exception:
            total_cost_m = None
            total_distance_only = None

        # Afficher la solution intermédiaire
        print("\n--- Solution intermédiaire ---")
        if total_distance_only is not None:
            print(f"Distance totale (objectif) : {total_distance_only / 1000:.2f} km")
        else:
            print("Distance totale (objectif) : N/A")
        print(f"Nombre de véhicules utilisés : {vehicles_used}")
        print("Tapez 's' puis Entrée pour STOPPER et garder la solution actuelle,")
        print("ou appuyez sur Entrée pour CONTINUER l'optimisation.")

        try:
            ans = input("Votre choix (s=stop, Entrée=continuer) : ").strip().lower()
        except Exception:
            ans = ''

        if ans == 's':
            routing_model.solver().FinishCurrentSearch()
            print("Arrêt demandé : retour de la meilleure solution actuelle.")
        else:
            print("Poursuite de la recherche...")

# Définir le callback de distance
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data['distance_matrix'][from_node][to_node] * 1000)  # Convertir km → m

distance_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(distance_callback_index)

# Définir le callback de temps
SERVICE_TIME = 5  # minutes par client

def time_callback(from_index, to_index):
    f = manager.IndexToNode(from_index)
    t = manager.IndexToNode(to_index)
    # temps trajet + service au point de départ
    return int(data['time_matrix'][f][t] + SERVICE_TIME)

def time_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data['time_matrix'][from_node][to_node] + SERVICE_TIME)

time_callback_index = routing.RegisterTransitCallback(time_callback)

routing.AddDimension(
    time_callback_index,
    0,     # pas de temps d'attente autorisé
    480,   # limite en minutes par véhicule
    True,  # commence à zéro
    'Time'
)

# Définir le callback de capacité
def demand_callback(from_index):
    return data['demands'][manager.IndexToNode(from_index)]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # pas de slack, c'est-à-dire que les véhicules ne peuvent pas dépasser leur capacité
    data['vehicle_capacities'],
    True,
    'Capacity'
)

# Ajouter un coût fixe par véhicule pour encourager l'utilisation de moins de véhicules 
# L'unité est la même que celle de l'arc-cost (ici mètres), donc 1_000_000 ≈ 1000 km
fixed_cost = 1_000_000  # ≈ 1000 km

for v in range(data['num_vehicles']):
    routing.SetFixedCostOfVehicle(fixed_cost, v)

# Callback interactif branché
solution_callback = InteractiveSolutionCallback(manager, routing, interval_seconds=60)
routing.AddAtSolutionCallback(solution_callback)

# Paramètres de recherche
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
search_parameters.log_search = True
search_parameters.time_limit.seconds = 300

# Résoudre
solution = routing.SolveWithParameters(search_parameters)

# Affichage des résultats
if solution:
    print("Solution trouvée :")
    total_distance = 0
    total_load = 0
    vehicles_used = 0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id) # Index de départ pour le véhicule qui est le dépôt
        route = []
        route_distance = 0
        route_load = 0
        time_dimension = routing.GetDimensionOrDie('Time')

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            client_id = clients_df.iloc[node_index]['id']
            route.append(str(client_id))
            route_load += data['demands'][node_index]
            previous_index = index # Index précédent pour calculer la distance ensuite
            index = solution.Value(routing.NextVar(index)) # Index suivant
            # route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
            from_node = manager.IndexToNode(previous_index)
            to_node = manager.IndexToNode(index)
            route_distance += int(data['distance_matrix'][from_node][to_node] * 1000)
        if len(route) > 1:
            vehicles_used += 1  # Compte le véhicule utilisé
            route.append("RETOUR_DEPOT")
            route_time = solution.Min(time_dimension.CumulVar(index))  # minutes totales
            print(f"Camion {vehicle_id + 1}: {' → '.join(route)} "
                f"(Distance: {round(route_distance / 1000, 2)} km, "
                f"Charge: {round(route_load / 1000, 2)} kg, "
                f"Temps: {route_time} min)")
            total_distance += route_distance
            total_load += route_load
    
    print(f"\nNombre de véhicules/camions utilisés: {vehicles_used}")
    print(f"Distance totale: {round(total_distance / 1000, 2)} km")
    print(f"Charge totale: {round(total_load / 1000, 2)} kg")
else:
    print("Aucune solution trouvée.")
    print("Statut du solveur:", routing.status())

I0000 00:00:1755382295.073226 3042459 search.cc:310] Start search (memory used = 1169.58 MB)
I0000 00:00:1755382295.096620 3042459 search.cc:310] Root node processed (time = 23 ms, constraints = 14206, memory used = 1173.64 MB)



--- Solution intermédiaire ---
Distance totale (objectif) : 1213.64 km
Nombre de véhicules utilisés : 32
Tapez 's' puis Entrée pour STOPPER et garder la solution actuelle,
ou appuyez sur Entrée pour CONTINUER l'optimisation.
Arrêt demandé : retour de la meilleure solution actuelle.
Solution trouvée :
Camion 5: DEPOT_CASA → C29883 → C25031 → C29864 → C25025 → C29139 → C20126 → C26076 → CN00785 → CM02288 → C25965 → CM02015 → CN00603 → C21522 → C23788 → CM21120 → C27090 → C25846 → M40005 → CM00848 → C20122 → C20136 → CM02655 → C26722 → C20324 → MH00155 → MH00002 → CM00804 → C25899 → C20233 → MH00166 → C20142 → C20232 → MH00222 → C20123 → C25885 → CN00797 → MH00223 → MH00249 → M40004 → C25890 → CN00663 → CN00988 → CM00779 → C25804 → M40046 → CN00506 → C22328 → MH00285 → CM02656 → CM02131 → C26204 → CM00851 → C25831 → CN00796 → CM02217 → M40048 → CM02099 → MH00284 → MH00039 → CN00385 → CM02657 → CM02697 → CM02215 → CM00638 → RETOUR_DEPOT (Distance: 98.92 km, Charge: 3022.43 kg, Temps: 444 

I0000 00:00:1755382330.713543 3042459 search.cc:310] Solution #0 (33213638, time = 35640 ms, branches = 34, failures = 1, depth = 33, memory used = 1212.89 MB, limit = 1%)
I0000 00:00:1755382330.735216 3042459 search.cc:310] End search (time = 35661 ms, branches = 34, failures = 2, memory used = 1211.91 MB, speed = 0 branches/s)


## Étape 4 : Sauvegarde des routes pour visualisation avec Folium

In [ ]:
import csv

# Liste pour stocker les segments de chaque route
route_lines = []

if solution:
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        
        # Vérifier d'abord si ce véhicule a une route réelle
        temp_index = index
        route_nodes = []
        while not routing.IsEnd(temp_index):
            node_index = manager.IndexToNode(temp_index)
            route_nodes.append(node_index)
            temp_index = solution.Value(routing.NextVar(temp_index))
        
        # Si le véhicule n'a qu'un seul noeud (le dépôt), on l'ignore
        if len(route_nodes) <= 1:
            continue
            
        # Si on arrive ici, le véhicule a une route réelle, on traite ses segments
        index = routing.Start(vehicle_id)
        previous_index = index
        
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            next_is_end = routing.IsEnd(index)
            
            if not next_is_end:
                next_node_index = manager.IndexToNode(index)

                # Coordonnées de la source (point actuel)
                src_lat = clients_df.loc[node_index, 'lat']
                src_lng = clients_df.loc[node_index, 'lng']
                
                # Coordonnées de la cible (prochain point)
                dst_lat = clients_df.loc[next_node_index, 'lat']
                dst_lng = clients_df.loc[next_node_index, 'lng']
                
                # Ajouter le segment à la liste
                route_lines.append({
                    'vehicle_id': vehicle_id + 1,
                    'source_lat': src_lat,
                    'source_lng': src_lng,
                    'target_lat': dst_lat,
                    'target_lng': dst_lng
                })
            else:
                # Dernier segment : retour au dépôt
                src_lat = clients_df.loc[node_index, 'lat']
                src_lng = clients_df.loc[node_index, 'lng']
                depot_lat = clients_df.loc[data['depot'], 'lat']
                depot_lng = clients_df.loc[data['depot'], 'lng']
                
                route_lines.append({
                    'vehicle_id': vehicle_id + 1,
                    'source_lat': src_lat,
                    'source_lng': src_lng,
                    'target_lat': depot_lat,
                    'target_lng': depot_lng
                })

# Sauvegarder dans routes.csv pour visualisation
with open('static/routes.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['vehicle_id', 'source_lat', 'source_lng', 'target_lat', 'target_lng'])
    writer.writeheader()
    writer.writerows(route_lines)

print(f"Fichier routes.csv généré avec succès pour {len(set(line['vehicle_id'] for line in route_lines))} véhicules utilisés.")

Fichier routes.csv généré avec succès pour 32 véhicules utilisés.


In [ ]:
import folium
import polyline
import itertools
import numpy as np
from folium.plugins import MarkerCluster

# Charger le fichier routes.csv
def load_routes():
    try:
        df = pd.read_csv('static/routes.csv')
        required_cols = {'vehicle_id', 'source_lat', 'source_lng', 'target_lat', 'target_lng'}
        if not required_cols.issubset(df.columns):
            raise ValueError(f"Colonnes manquantes : {required_cols - set(df.columns)}")
        return df
    except Exception as e:
        print(f"Erreur chargement fichier: {e}")
        exit()

routes_df = load_routes()
print("Données chargées depuis routes.csv")

# Nettoyage
routes_df_clean = routes_df.dropna(subset=['source_lat', 'source_lng', 'target_lat', 'target_lng'])

# Fonction OSRM pour tracer les routes réelles
def get_route_coords(start, end, max_retries=3):
    for _ in range(max_retries):
        try:
            url = f"http://localhost:5001/route/v1/driving/{start[1]},{start[0]};{end[1]},{end[0]}?overview=full&geometries=polyline"
            response = requests.get(url, timeout=10).json()
            if response.get('code') == 'Ok':
                return polyline.decode(response['routes'][0]['geometry'])
        except Exception:
            continue
    return None

# Initialisation de la carte Folium
first_row = routes_df_clean.iloc[0]
start_lat, start_lng = first_row['source_lat'], first_row['source_lng']

m = folium.Map(location=[start_lat, start_lng], zoom_start=12, tiles='cartodbpositron')

# Marqueur du dépôt
folium.Marker(
    location=[start_lat, start_lng],
    popup="🏭 Dépôt",
    icon=folium.Icon(color='black', icon='home', prefix='fa')
).add_to(m)

# Tracer les routes pour chaque véhicule
vehicle_ids = routes_df_clean['vehicle_id'].unique()

icon_colors = [
    'red', 'blue', 'green', 'orange',
    'purple', 'darkred', 'darkblue', 'darkgreen',
    'pink', 'lightblue', 'lightgreen', 'beige',
    'cadetblue', 'gray', 'black', 'lightgray',
]
colors = itertools.cycle(icon_colors)

cluster = MarkerCluster(name="Camions").add_to(m)

for vid in vehicle_ids:
    try:
        color = next(colors)
        fg = folium.FeatureGroup(name=f"Camion {vid}", show=True)

        vehicle_df = routes_df_clean[routes_df_clean['vehicle_id'] == vid]

        for _, row in vehicle_df.iterrows():
            src = (row['source_lat'], row['source_lng'])
            dst = (row['target_lat'], row['target_lng'])

            if np.isnan(src[0]) or np.isnan(src[1]) or np.isnan(dst[0]) or np.isnan(dst[1]):
                continue

            segment = get_route_coords(src, dst)
            if segment:
                folium.PolyLine(segment, color=color, weight=4, opacity=0.8,
                                popup=f"Camion {vid}").add_to(fg)

            # Marqueurs source et destination
            folium.CircleMarker(location=src, radius=4, color=color, fill=True, fill_opacity=0.7).add_to(fg)
            folium.CircleMarker(location=dst, radius=4, color=color, fill=True, fill_opacity=0.7).add_to(fg)

        # Ajouter l'icône camion au départ
        first_point = vehicle_df.iloc[0]
        folium.Marker(
            [first_point['source_lat'], first_point['source_lng']],
            icon=folium.Icon(color=color, icon='truck', prefix='fa'),
            popup=f"<b>Camion {vid}</b>"
        ).add_to(cluster)

        m.add_child(fg)
        print(f"Camion {vid} tracé")

    except Exception as e:
        print(f"Erreur pour Camion {vid} : {e}")
        continue

# Affichage de la carte
folium.LayerControl(collapsed=False).add_to(m)
m.save('carte/routes_folium.html')
print("Carte Folium enregistrée sous 'routes_folium.html'")


Données chargées depuis routes.csv
Camion 5 tracé
Camion 6 tracé
Camion 7 tracé
Camion 8 tracé
Camion 9 tracé
Camion 10 tracé
Camion 11 tracé
Camion 12 tracé
Camion 13 tracé
Camion 14 tracé
Camion 15 tracé
Camion 16 tracé
Camion 17 tracé
Camion 18 tracé
Camion 19 tracé
Camion 20 tracé
Camion 21 tracé
Camion 22 tracé
Camion 23 tracé
Camion 24 tracé
Camion 25 tracé
Camion 26 tracé
Camion 27 tracé
Camion 28 tracé
Camion 29 tracé
Camion 30 tracé
Camion 31 tracé
Camion 32 tracé
Camion 33 tracé
Camion 34 tracé
Camion 35 tracé
Camion 36 tracé


FileNotFoundError: [Errno 2] No such file or directory: 'carte/routes_folium.html'